In [1]:
#building damage Classification Prediction Model
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import DistanceMetric
import pickle

#reading the dataset to python 
df=pd.read_csv("csv_building_structure.csv") 
# Handling Missing values
df.isna().sum()
index=[83766, 131558, 131579, 131627, 131629, 131640, 131652, 131654,131655, 131656, 131929, 131932]
for i in index:
    df.drop(i,axis=0,inplace=True)
    
def impute_plinth_area(cols):
    rtype=cols[0]
    plinth=cols[1]
    if pd.isnull(plinth):
        if rtype == "Bamboo/Timber-Heavy roof" :
            return 360
        elif rtype == "Bamboo/Timber-Light roof" :
            return 345
        elif rtype == "RCC/RB/RBC" :
            return 700
    else:
        return plinth

df["roof_type"]=df["roof_type"].fillna(df["roof_type"].mode()[0])
df["plinth_area_sq_ft"]=df[["roof_type","plinth_area_sq_ft"]].apply(impute_plinth_area,axis=1)

#Removing the duplicates
df.drop_duplicates(keep='first',inplace=True)

#Handling of outlirers in the dataset
Q1=df['age_building'].quantile(0.25)
Q3=df['age_building'].quantile(0.75)
IQR=Q3-Q1    
LL=Q1-(1.5*IQR)
UL=Q3+(1.5*IQR)
df['age_building']= np.where(df['age_building']>UL,UL,np.where(df['age_building']<LL,LL,
                                                                           df['age_building']))
Raw_Data=df.copy()
Raw_Data['plinth_area_sq_ft'] = np.log(Raw_Data['plinth_area_sq_ft'])
Raw_Data['height_ft_pre_eq'] = np.log(Raw_Data['height_ft_pre_eq'])
Raw_Data.drop(['building_id', 'district_id', 'vdcmun_id', 'ward_id'],axis=1,inplace=True)
Raw_Data.drop(['has_superstructure_timber',
       'plan_configuration', 'has_superstructure_cement_mortar_brick',
       'has_superstructure_bamboo', 'has_superstructure_stone_flag',
       'has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_rc_non_engineered', 'has_superstructure_other',
       'has_superstructure_rc_engineered'],axis=1,inplace=True)

#Encoding the Target column
lbl=LabelEncoder()
Raw_Data["damage_grade"]=lbl.fit_transform(Raw_Data["damage_grade"])

X = Raw_Data.drop(['damage_grade'],axis=1)
y= Raw_Data['damage_grade']
 
import category_encoders as ce 
# # Define catboost encoder
cbe_encoder = ce.cat_boost.CatBoostEncoder()
  
# # Fit encoder and transform the features
cbe_encoder.fit(X, y)
X = cbe_encoder.transform(X)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=34,stratify=y)  
std=StandardScaler()
X_train=std.fit_transform(X_train)
X_test=std.transform(X_test)

# from imblearn.over_sampling import SMOTE
# oversample = SMOTE()
# X_train, y_train = oversample.fit_resample(X_train, y_train)

# classifier=RandomForestClassifier()
# classifier.fit(X_train,y_train)

# # # Saving the model to disk
# pickle.dump(classifier, open("model.pkl","wb"))


In [2]:
X

,count_floors_pre_eq,age_building,plinth_area_sq_ft,height_ft_pre_eq,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,has_superstructure_mud_mortar_stone
0,1,9.0,5.662960,2.197225,2.598719,2.169431,2.720530,2.807554,1.995678,2.599072,1
1,1,15.0,5.897154,2.197225,2.598719,2.169431,2.720530,2.807554,1.995678,2.599072,1
2,1,20.0,5.950643,2.197225,2.598719,2.169431,2.720530,2.807554,1.995678,2.599072,1
3,1,20.0,5.743003,2.197225,2.598719,2.169431,2.720530,2.807554,1.995678,2.599072,1
4,1,30.0,5.730100,2.197225,2.598719,2.169431,2.720530,2.807554,1.995678,2.599072,1
...,...,...,...,...,...,...,...,...,...,...,...
762101,2,55.5,5.105945,2.890372,2.598719,2.893527,2.775768,2.807554,2.720470,2.224853,1
762102,2,35.0,5.834811,2.890372,2.598719,2.893527,2.775768,2.807554,2.720470,2.776982,1
762103,2,35.0,5.834811,2.890372,2.598719,2.893527,2.775768,2.807554,2.720470,2.599072,1
762104,2,19.0,5.723585,2.890372,2.598719,2.893527,2.775768,2.807554,2.720470,2.599072,1


In [3]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.under_sampling import OneSidedSelection 


In [4]:
print('Original dataset shape {}'.format(Counter(y)))


Original dataset shape Counter({4: 253777, 3: 175797, 2: 131894, 1: 85458, 0: 76598})


In [5]:
oss = OneSidedSelection(random_state=42)
X_res, y_res = oss.fit_resample(X, y)
print('Resampled dataset shape %s' % Counter(y_res))

Resampled dataset shape Counter({4: 210629, 3: 137073, 2: 97435, 0: 76598, 1: 61167})


In [6]:
X_train,X_test,y_train,y_test = train_test_split(X_res,y_res,test_size=0.3,random_state=34,stratify=y_res)

In [ ]:
from sklearn.preprocessing import StandardScaler

std=StandardScaler()
X_train=std.fit_transform(X_train)
X_test=std.transform(X_test)

In [10]:
classifier=RandomForestClassifier()
classifier.fit(X_train,y_train)

RandomForestClassifier()

In [13]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [13]:
y_pred=classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[13415  2739  2102  1808  2915]
 [ 4420  3100  2853  3103  4874]
 [ 2905  2524  6156  6947 10699]
 [ 1710  2184  5946 13237 18045]
 [ 1851  2581  6857 13722 38178]]
0.42366087001275227
              precision    recall  f1-score   support

           0       0.55      0.58      0.57     22979
           1       0.24      0.17      0.20     18350
           2       0.26      0.21      0.23     29231
           3       0.34      0.32      0.33     41122
           4       0.51      0.60      0.55     63189

    accuracy                           0.42    174871
   macro avg       0.38      0.38      0.38    174871
weighted avg       0.41      0.42      0.41    174871



In [14]:
# Repeated Edited Neibours

In [7]:
print('Original dataset shape {}'.format(Counter(y)))

Original dataset shape Counter({4: 253777, 3: 175797, 2: 131894, 1: 85458, 0: 76598})


In [8]:
from imblearn.under_sampling import RepeatedEditedNearestNeighbours # doctest : +NORMALIZE_WHITESPACE

renn = RepeatedEditedNearestNeighbours()
X_res, y_res = renn.fit_resample(X, y)
print('Resampled dataset shape %s' % Counter(y_res))

Resampled dataset shape Counter({0: 76598, 4: 32693, 3: 6553, 2: 2138, 1: 1439})


In [9]:
X_train,X_test,y_train,y_test = train_test_split(X_res,y_res,test_size=0.3,random_state=34,stratify=y_res)

In [10]:
from sklearn.preprocessing import StandardScaler

std=StandardScaler()
X_train=std.fit_transform(X_train)
X_test=std.transform(X_test)

In [11]:
classifier=RandomForestClassifier()
classifier.fit(X_train,y_train)

RandomForestClassifier()

In [14]:
y_pred=classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[21261    54    74   258  1333]
 [  272   155     1     0     4]
 [  225     1   327    20    68]
 [  301     0    19  1199   447]
 [  686     1    17   133  8971]]
0.8907527842130237
              precision    recall  f1-score   support

           0       0.93      0.93      0.93     22980
           1       0.73      0.36      0.48       432
           2       0.75      0.51      0.61       641
           3       0.74      0.61      0.67      1966
           4       0.83      0.91      0.87      9808

    accuracy                           0.89     35827
   macro avg       0.80      0.66      0.71     35827
weighted avg       0.89      0.89      0.89     35827



In [20]:
# Condensed Nearest

In [22]:

from imblearn.under_sampling import CondensedNearestNeighbour 
 

In [23]:
print('Original dataset shape {}'.format(Counter(y)))

Original dataset shape Counter({4: 253777, 3: 175797, 2: 131894, 1: 85458, 0: 76598})


In [ ]:
cnn = CondensedNearestNeighbour(random_state=42) 
X_res, y_res = cnn.fit_resample(X, y) 
print('Resampled dataset shape %s' % Counter(y_res)) 

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_res,y_res,test_size=0.3,random_state=34,stratify=y_res)

In [ ]:
classifier=RandomForestClassifier()
classifier.fit(X_train,y_train)

In [ ]:
y_pred=classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [27]:
Raw_Data.columns

Index(['count_floors_pre_eq', 'age_building', 'plinth_area_sq_ft',
       'height_ft_pre_eq', 'land_surface_condition', 'foundation_type',
       'roof_type', 'ground_floor_type', 'other_floor_type', 'position',
       'has_superstructure_mud_mortar_stone', 'damage_grade'],
      dtype='object')

In [28]:
Raw_Data

,count_floors_pre_eq,age_building,plinth_area_sq_ft,height_ft_pre_eq,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,has_superstructure_mud_mortar_stone,damage_grade
0,1,9.0,5.662960,2.197225,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,1,2
1,1,15.0,5.897154,2.197225,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,1,4
2,1,20.0,5.950643,2.197225,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,1,1
3,1,20.0,5.743003,2.197225,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,1,1
4,1,30.0,5.730100,2.197225,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
762101,2,55.5,5.105945,2.890372,Flat,Mud mortar-Stone/Brick,Bamboo/Timber-Heavy roof,Mud,Timber-Planck,Attached-2 side,1,4
762102,2,35.0,5.834811,2.890372,Flat,Mud mortar-Stone/Brick,Bamboo/Timber-Heavy roof,Mud,Timber-Planck,Attached-1 side,1,4
762103,2,35.0,5.834811,2.890372,Flat,Mud mortar-Stone/Brick,Bamboo/Timber-Heavy roof,Mud,Timber-Planck,Not attached,1,4
762104,2,19.0,5.723585,2.890372,Flat,Mud mortar-Stone/Brick,Bamboo/Timber-Heavy roof,Mud,Timber-Planck,Not attached,1,4


In [30]:
X["roof_type"].value_counts()

2.720530    477916
2.775768    201734
0.720128     43874
Name: roof_type, dtype: int64

In [51]:
Raw_Data["has_superstructure_mud_mortar_stone"].value_counts()

1    574546
0    148978
Name: has_superstructure_mud_mortar_stone, dtype: int64

In [52]:
X["has_superstructure_mud_mortar_stone"].value_counts()

1    574546
0    148978
Name: has_superstructure_mud_mortar_stone, dtype: int64